In [ ]:
import nltk
nltk.download('punkt')
import json
import pickle
import numpy as np
import nltk
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from nltk.stem import SnowballStemmer


import json
import pickle
import numpy as np
import nltk
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish') 
palabras_ignoradas = ["?","¿","!","¡"]
archivo_datos = open("intents.json").read()
intenciones = json.loads(archivo_datos)

def tokenizador():
    palabras = []
    categorias = []
    documentos = []

    for intencion in intenciones["intenciones"]:
        for patron in intencion["patterns"]:
            w = nltk.word_tokenize(patron)
            palabras.extend(w)
            documentos.append((w, intencion["categoria"])) 
            if intencion["categoria"] not in categorias: 
                categorias.append(intencion["categoria"]) 

    return palabras, categorias, documentos

def lematizador(palabras, categorias, documentos):
    palabras = [stemmer.stem(w.lower()) for w in palabras if w not in palabras_ignoradas]
    palabras2 = palabras
    pickle.dump(palabras, open("palabras.pkl", "wb"))
    pickle.dump(categorias, open("categorias.pkl", "wb"))
    return palabras2

def entrenamiento(palabras, categorias, documentos):
    entrenamiento = []
    salida_vacia = [0] * len(categorias)
    for doc in documentos:
        bolsa = []
        palabras_patron = doc[0]
        palabras_patron = [stemmer.stem(palabra.lower()) for palabra in palabras_patron if palabra not in palabras_ignoradas]
        for w in palabras:
            bolsa.append(1) if w in palabras_patron else bolsa.append(0)
        salida_fila = list(salida_vacia)
        salida_fila[categorias.index(doc[1])] = 1
        entrenamiento.append([bolsa, salida_fila])
    entrenamiento = np.array(entrenamiento, dtype=object)
    x_entrenamiento = list(entrenamiento[:,0])
    y_entrenamiento = list(entrenamiento[:,1])
    return x_entrenamiento, y_entrenamiento

def construir_modelo(x_entrenamiento, y_entrenamiento, categorias):
    modelo = Sequential()
    modelo.add(Dense(128, input_shape=(len(x_entrenamiento[0]),), activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dropout(0.5))
    modelo.add(Dense(len(categorias), activation='softmax'))
    modelo.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    historial = modelo.fit(np.array(x_entrenamiento), np.array(y_entrenamiento), epochs=50, batch_size=5, verbose=1, validation_split=0.16)

    import matplotlib.pyplot as plt

    # Plot training & validation accuracy values
    plt.plot(historial.history['accuracy'])
    plt.plot(historial.history['val_accuracy'])
    plt.title('Modelo de precisión')
    plt.ylabel('Precisión')
    plt.xlabel('Epoca')
    plt.legend(['Entrenamiento', 'Prueba'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(historial.history['loss'])
    plt.plot(historial.history['val_loss'])
    plt.title('Modelo de pérdida')
    plt.ylabel('Pérdida')
    plt.xlabel('Epoca')
    plt.legend(['Entrenamiento', 'Prueba'], loc='upper left')
    plt.show()

    return modelo

palabras, categorias, documentos = tokenizador()
palabras = lematizador(palabras, categorias, documentos)
x_entrenamiento, y_entrenamiento = entrenamiento(palabras, categorias, documentos)
modelo = construir_modelo(x_entrenamiento, y_entrenamiento, categorias)
